In [13]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import numpy as np
from pyspark.sql.functions import *
import pandas as pd
# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Tutorial 1")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)

In [9]:

merchant_fraud = spark.read.option("header",True).csv('../data/tables/merchant_fraud_probability.csv')
tbl_merchants = spark.read.parquet('../data/tables/tbl_merchants.parquet')


In [7]:
merchant_fraud = merchant_fraud.select(col("merchant_abn"),col("fraud_probability"),to_date(col("order_datetime"),"yyyy-MM-dd").alias("date"))

print(merchant_fraud.count())
merchant_fraud.filter("date > '2021-02-28'")
merchant_fraud.filter("date < '2022-08-28'")
print(merchant_fraud.count())

114
114


In [8]:
merchant_fraud_group = merchant_fraud.groupBy('merchant_abn').agg(F.count('fraud_probability').alias('fraud_count_abn'))
merchant_fraud_group.limit(2)

merchant_abn,fraud_count_abn
67264251405,1
43083074133,1


In [14]:
tbl_merchants_pd = tbl_merchants.toPandas()
for i in range(int(tbl_merchants_pd['tags'].count())):
    tbl_merchants_pd['tags'].iloc[i] = tbl_merchants_pd['tags'].iloc[i].replace(r'[', r'(').replace(r']', r')')

/var/folders/4w/p31d1t_x5pqbppcsm7p9kxt00000gn/T/ipykernel_34251/2153331469.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tbl_merchants_pd['tags'].iloc[i] = tbl_merchants_pd['tags'].iloc[i].replace(r'[', r'(').replace(r']', r')')


In [16]:
merchant_tags = tbl_merchants_pd['tags'].str.split(')', expand=True)

for row in range(int(len(merchant_tags))):
    for col in range(3):
        merchant_tags.iloc[row,col] = merchant_tags.iloc[row,col].replace(r'((', r'').replace(r', (', r'').replace(r'take rate:', r'')
merchant_tags.rename(columns = {0 : 'Store type', 1 : 'Revenue levels', 2 : 'Take rate'}, inplace = True)
merchant_tags = merchant_tags[['Store type', 'Revenue levels', 'Take rate']]

In [22]:
tbl_merchants_pd[['Store type', 'Revenue levels', 'Take rate']] = merchant_tags[['Store type', 'Revenue levels', 'Take rate']]
#merchant.drop(columns=['tags'])
tbl_merchants_pd['Store type'] = tbl_merchants_pd['Store type'].str.lower()

for i in range(len(tbl_merchants_pd)):
    tbl_merchants_pd['Store type'][i] = ' '.join(tbl_merchants_pd['Store type'][i].split())
 

/var/folders/4w/p31d1t_x5pqbppcsm7p9kxt00000gn/T/ipykernel_34251/3875288317.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tbl_merchants_pd['Store type'][i] = ' '.join(tbl_merchants_pd['Store type'][i].split())


In [34]:
merchants = tbl_merchants_pd.drop(columns = ['name','tags'])
merchants.to_parquet(f"../data/curated/final_merchants.parquet")